In [1]:
import datetime
import pandas as pd
import numpy as np

In [22]:
wind = pd.read_csv('../Classification Data/wind.csv')
wind.pop('Unnamed: 0')
wind.tail()

,ID,Name,Time,Record ID,Status,Latitude,Longitude,Max wind (knots),Min pressure (mbar),34kt wind radii max NE,...,34kt wind radii max SW,34kt wind radii max NW,50kt wind radii max NE,50kt wind radii max SE,50kt wind radii max SW,50kt wind radii max NW,64kt wind radii max NE,64kt wind radii max SE,64kt wind radii max SW,64kt wind radii max NW
51341,AL162018,OSCAR,2018-11-03 12:00:00,NaN,EX,57.9N,19.6W,55,960.0,780.0,...,660.0,480.0,0.0,240.0,0.0,0.0,0.0,0.0,0.0,0.0
51342,AL162018,OSCAR,2018-11-03 18:00:00,NaN,EX,58.9N,17.1W,50,964.0,480.0,...,660.0,420.0,0.0,240.0,0.0,0.0,0.0,0.0,0.0,0.0
51343,AL162018,OSCAR,2018-11-04 00:00:00,NaN,EX,59.8N,14.5W,45,968.0,360.0,...,480.0,360.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51344,AL162018,OSCAR,2018-11-04 06:00:00,NaN,EX,60.8N,12.1W,40,973.0,270.0,...,360.0,270.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51345,AL162018,OSCAR,2018-11-04 12:00:00,NaN,EX,62.4N,9.1W,40,977.0,240.0,...,0.0,210.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
image_path = '../Image Data/pics/'

labels = pd.DataFrame({'Filename':[], 'Category':[]})

delta = datetime.timedelta(hours = 3)
last = datetime.datetime(2018, 12, 1, 0)
frame = datetime.datetime(2018, 6, 1, 0)

def category(time):
    rows = wind.shape[0]
    current_row = int(rows/2)
    current_time = wind.iloc[current_row, 2]
    
    difference = time - current_time
    print()

category(datetime())


1960-07-28 12:00:00


In [43]:
while frame < last:
    filepath = image_path + frame.strftime("%Y-%m-%d-%H") + ".jpg"
    try:
        with open(filepath) as image:
            pass
        row = pd.DataFrame({'Filename': [frame], 'Category': [0]})
        labels = labels.append(row)
    except:
        pass
    frame += delta

print(wind.loc[wind['Time']==datetime.datetime(2018, 7, 4, hour=12)].index)

labels.head()

Int64Index([], dtype='int64')


,Filename,Category
0,2018-06-01 00:00:00,0.0
0,2018-06-01 03:00:00,0.0
0,2018-06-01 06:00:00,0.0
0,2018-06-01 09:00:00,0.0
0,2018-06-01 12:00:00,0.0
